## Задание 1  
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = [‘python’, ‘парсинг’]

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [2]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

KEYWORDS = ['python', 'парсинг', 'challenge', 'разработка', 'фольклор', 'верификация']

all_news = pd.DataFrame()

res = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(res.text, 'html.parser')
posts = soup.find_all('article', class_='tm-articles-list__item')
for post in posts:
    time.sleep(0.3)
    post_lower = post.text.lower()
    for word in KEYWORDS:
        if word in post_lower:
            if post.find('div', class_='tm-article-snippet') is not None:
                date = pd.to_datetime(post.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime'), dayfirst=True).date()
                title = post.find('h2', class_ = 'tm-article-snippet__title tm-article-snippet__title_h2').text
                link = 'https://habr.com' + post.find('a', class_ = 'tm-article-snippet__title-link').get('href')
                row = {'title' : title, 'date' : date, 'link': link}
                all_news = all_news.append(row, ignore_index = True)
                break
            elif post.find('div', class_='tm-megapost-snippet') is not None:
                date = pd.to_datetime(post.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date').find('time').get('datetime'), dayfirst=True).date()
                title = post.find('h2', class_ = 'tm-megapost-snippet__title').text
                link = 'https://habr.com' + post.find('a', class_ = 'tm-megapost-snippet__link tm-megapost-snippet__date').get('href')
                row = {'title' : title, 'date' : date, 'link': link}
                all_news = all_news.append(row, ignore_index = True)
                break
                
all_news

# Jupiter Notebook выводит ключи словаря в произвольном порядке, поэтому сохранить порядок столбцов, как в задании, не получается
# В Google Colab такой проблемы нет

,date,link,title
0,2021-08-11,https://habr.com/ru/company/microsoft/blog/572...,Квантовая разработка с использованием QDK
1,2021-08-11,https://habr.com/ru/company/jugru/blog/570220/,Взлом и внедрение своего кода в чужое iOS-прил...
2,2021-08-11,https://habr.com/ru/company/skillfactory/blog/...,Знакомство с Panda Gym
3,2021-08-11,https://habr.com/ru/post/572392/,Особенности создания клона Minecraft «DScraft»...
4,2021-08-11,https://habr.com/ru/company/dododev/blog/572216/,В шорткаты умеешь? А если научу?
5,2021-08-11,https://habr.com/ru/post/572390/,Личный кабинет: обрабатываем жалобы клиентов B...
6,2021-08-11,https://habr.com/ru/post/572388/,Язык сетевого программирования P4. Часть 1: об...
7,2021-08-11,https://habr.com/ru/company/timeweb/blog/571916/,Civilization Сида Мейера: баг «ядерного Ганди»...
8,2021-08-11,https://habr.com/ru/company/vsk_insurance/blog...,"Автоботы, трансформируемся: как быстро и недор..."


## Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

In [14]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import json

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

payload = { 'emailAddresses': ['xxx@ya.ru', 'yyy@y.com', 'xxx@ya.ru', 'a@a.com']}
HEADERS = {
     'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
     'Vaar-Header-App-Build-Version': '1.0',
     'Vaar-Version': '0'
 }
res = requests.post(URL, headers= HEADERS, json =payload)
print(res.text)
result = res.json()['breaches'] 
pd.DataFrame(res.json()['breaches'])

{"breaches":{"37398":{"breachId":37398,"site":"yam.com","recordsCount":16322233,"description":"In 2013, the Taiwanese news website Yam was allegedly breached. The stolen data contains usernames, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.","publishDate":"2021-05-27T00:00:00Z","statistics":{"usernames":16322217,"passwords":16305095,"emails":15862137}},"37428":{"breachId":37428,"site":"pearldrummersforums.com","recordsCount":197776,"description":"At an unknown date, the online percussion site Pearl Drummers Forum was allegedly breached. The stolen data contains passwords, usernames, salts, IPs, email addresses and additional personal information. This breach is being privately shared on the internet.","publishDate":"2021-05-13T00:00:00Z","statistics":{"usernames":197675,"passwords":197768,"emails":197620}},"37458":{"breachId":37458,"site":"descomplica.com.br","recordsCount":4779635,"description":"In March 2021, th

,37398,37428,37458,37502,37504,37509,37510,37540,37546,37550,...,37606,37607,37632,37654,37690,37696,37785,37826,37827,37836
breachId,37398,37428,37458,37502,37504,37509,37510,37540,37546,37550,...,37606,37607,37632,37654,37690,37696,37785,37826,37827,37836
site,yam.com,pearldrummersforums.com,descomplica.com.br,swordfantasy.com,neo24.pl,printi.com.br,ajarn.com,wibdesign.de,piggyback.com,eduever.com,...,arenajunkies.com,frenchrosa.com,offnews.bg,patari.pk,forums.softballfans.com,articoolo.com,unknown,mymedicarequestion.com,jamtangan.com,forums.mypirata.me
recordsCount,16322233,197776,4779635,3202149,699115,563940,541191,250505,449792,187506,...,1151635,38200,172352,273320,109008,337166,4771990,9447,465122,334332
description,"In 2013, the Taiwanese news website Yam was al...","At an unknown date, the online percussion site...","In March 2021, the Brazilian online education ...","At an unknown date, the forum for SwordFantasy...","At an unknown date, the Polish online electron...","At an unknown date, the Brazilian B2B printing...","At an unknown date, the Thai English learning ...","At an unconfirmed date, the German web design ...","In July 2020, an online list of users of the g...","At an unknown date, the Korean education websi...",...,"At an unknown date, the World of Warcraft stra...","At an unknown date, the French online furnitur...","At an unknown date, the Bulgarian news site OF...","In June 2021, the music site Patari was allege...","In May 2021, the online softball discussion fo...","In May 2021, the content creation service Arti...",The proliferation of stolen or leaked-breach d...,"At an unknown date, the My Medicare Question (...","In July 2021, the Indonesian online watch stor...","In December 2019, the online video game forum ..."
publishDate,2021-05-27T00:00:00Z,2021-05-13T00:00:00Z,2021-05-13T00:00:00Z,2021-05-20T00:00:00Z,2021-06-30T00:00:00Z,2021-05-20T00:00:00Z,2021-06-10T00:00:00Z,2021-07-29T00:00:00Z,2021-05-20T00:00:00Z,2021-05-27T00:00:00Z,...,2021-06-10T00:00:00Z,2021-08-05T00:00:00Z,2021-06-24T00:00:00Z,2021-07-29T00:00:00Z,2021-08-05T00:00:00Z,2021-06-30T00:00:00Z,2021-07-15T00:00:00Z,2021-07-29T00:00:00Z,2021-07-22T00:00:00Z,2021-08-05T00:00:00Z
statistics,"{'usernames': 16322217, 'passwords': 16305095,...","{'usernames': 197675, 'passwords': 197768, 'em...","{'usernames': 2698, 'passwords': 4733278, 'ema...","{'usernames': 3198793, 'passwords': 3202109, '...","{'usernames': 111577, 'passwords': 697834, 'em...","{'usernames': 121453, 'passwords': 563927, 'em...","{'usernames': 10, 'passwords': 266358, 'emails...","{'usernames': 32521, 'passwords': 250505, 'ema...","{'usernames': 0, 'passwords': 449792, 'emails'...","{'usernames': 0, 'passwords': 179736, 'emails'...",...,"{'usernames': 1149613, 'passwords': 1108013, '...","{'usernames': 0, 'passwords': 16299, 'emails':...","{'usernames': 156123, 'passwords': 172352, 'em...","{'usernames': 271107, 'passwords': 104932, 'em...","{'usernames': 108984, 'passwords': 0, 'emails'...","{'usernames': 336911, 'passwords': 337164, 'em...","{'usernames': 255666, 'passwords': 4758916, 'e...","{'usernames': 0, 'passwords': 397, 'emails': 9...","{'usernames': 0, 'passwords': 432409, 'emails'...","{'usernames': 273666, 'passwords': 334332, 'em..."


In [15]:
pd.DataFrame(res.json()['breaches'])

,37398,37428,37458,37502,37504,37509,37510,37540,37546,37550,...,37606,37607,37632,37654,37690,37696,37785,37826,37827,37836
breachId,37398,37428,37458,37502,37504,37509,37510,37540,37546,37550,...,37606,37607,37632,37654,37690,37696,37785,37826,37827,37836
site,yam.com,pearldrummersforums.com,descomplica.com.br,swordfantasy.com,neo24.pl,printi.com.br,ajarn.com,wibdesign.de,piggyback.com,eduever.com,...,arenajunkies.com,frenchrosa.com,offnews.bg,patari.pk,forums.softballfans.com,articoolo.com,unknown,mymedicarequestion.com,jamtangan.com,forums.mypirata.me
recordsCount,16322233,197776,4779635,3202149,699115,563940,541191,250505,449792,187506,...,1151635,38200,172352,273320,109008,337166,4771990,9447,465122,334332
description,"In 2013, the Taiwanese news website Yam was al...","At an unknown date, the online percussion site...","In March 2021, the Brazilian online education ...","At an unknown date, the forum for SwordFantasy...","At an unknown date, the Polish online electron...","At an unknown date, the Brazilian B2B printing...","At an unknown date, the Thai English learning ...","At an unconfirmed date, the German web design ...","In July 2020, an online list of users of the g...","At an unknown date, the Korean education websi...",...,"At an unknown date, the World of Warcraft stra...","At an unknown date, the French online furnitur...","At an unknown date, the Bulgarian news site OF...","In June 2021, the music site Patari was allege...","In May 2021, the online softball discussion fo...","In May 2021, the content creation service Arti...",The proliferation of stolen or leaked-breach d...,"At an unknown date, the My Medicare Question (...","In July 2021, the Indonesian online watch stor...","In December 2019, the online video game forum ..."
publishDate,2021-05-27T00:00:00Z,2021-05-13T00:00:00Z,2021-05-13T00:00:00Z,2021-05-20T00:00:00Z,2021-06-30T00:00:00Z,2021-05-20T00:00:00Z,2021-06-10T00:00:00Z,2021-07-29T00:00:00Z,2021-05-20T00:00:00Z,2021-05-27T00:00:00Z,...,2021-06-10T00:00:00Z,2021-08-05T00:00:00Z,2021-06-24T00:00:00Z,2021-07-29T00:00:00Z,2021-08-05T00:00:00Z,2021-06-30T00:00:00Z,2021-07-15T00:00:00Z,2021-07-29T00:00:00Z,2021-07-22T00:00:00Z,2021-08-05T00:00:00Z
statistics,"{'usernames': 16322217, 'passwords': 16305095,...","{'usernames': 197675, 'passwords': 197768, 'em...","{'usernames': 2698, 'passwords': 4733278, 'ema...","{'usernames': 3198793, 'passwords': 3202109, '...","{'usernames': 111577, 'passwords': 697834, 'em...","{'usernames': 121453, 'passwords': 563927, 'em...","{'usernames': 10, 'passwords': 266358, 'emails...","{'usernames': 32521, 'passwords': 250505, 'ema...","{'usernames': 0, 'passwords': 449792, 'emails'...","{'usernames': 0, 'passwords': 179736, 'emails'...",...,"{'usernames': 1149613, 'passwords': 1108013, '...","{'usernames': 0, 'passwords': 16299, 'emails':...","{'usernames': 156123, 'passwords': 172352, 'em...","{'usernames': 271107, 'passwords': 104932, 'em...","{'usernames': 108984, 'passwords': 0, 'emails'...","{'usernames': 336911, 'passwords': 337164, 'em...","{'usernames': 255666, 'passwords': 4758916, 'e...","{'usernames': 0, 'passwords': 397, 'emails': 9...","{'usernames': 0, 'passwords': 432409, 'emails'...","{'usernames': 273666, 'passwords': 334332, 'em..."


In [16]:
res2 = pd.DataFrame(res.json()['breaches'])
res2

,37398,37428,37458,37502,37504,37509,37510,37540,37546,37550,...,37606,37607,37632,37654,37690,37696,37785,37826,37827,37836
breachId,37398,37428,37458,37502,37504,37509,37510,37540,37546,37550,...,37606,37607,37632,37654,37690,37696,37785,37826,37827,37836
site,yam.com,pearldrummersforums.com,descomplica.com.br,swordfantasy.com,neo24.pl,printi.com.br,ajarn.com,wibdesign.de,piggyback.com,eduever.com,...,arenajunkies.com,frenchrosa.com,offnews.bg,patari.pk,forums.softballfans.com,articoolo.com,unknown,mymedicarequestion.com,jamtangan.com,forums.mypirata.me
recordsCount,16322233,197776,4779635,3202149,699115,563940,541191,250505,449792,187506,...,1151635,38200,172352,273320,109008,337166,4771990,9447,465122,334332
description,"In 2013, the Taiwanese news website Yam was al...","At an unknown date, the online percussion site...","In March 2021, the Brazilian online education ...","At an unknown date, the forum for SwordFantasy...","At an unknown date, the Polish online electron...","At an unknown date, the Brazilian B2B printing...","At an unknown date, the Thai English learning ...","At an unconfirmed date, the German web design ...","In July 2020, an online list of users of the g...","At an unknown date, the Korean education websi...",...,"At an unknown date, the World of Warcraft stra...","At an unknown date, the French online furnitur...","At an unknown date, the Bulgarian news site OF...","In June 2021, the music site Patari was allege...","In May 2021, the online softball discussion fo...","In May 2021, the content creation service Arti...",The proliferation of stolen or leaked-breach d...,"At an unknown date, the My Medicare Question (...","In July 2021, the Indonesian online watch stor...","In December 2019, the online video game forum ..."
publishDate,2021-05-27T00:00:00Z,2021-05-13T00:00:00Z,2021-05-13T00:00:00Z,2021-05-20T00:00:00Z,2021-06-30T00:00:00Z,2021-05-20T00:00:00Z,2021-06-10T00:00:00Z,2021-07-29T00:00:00Z,2021-05-20T00:00:00Z,2021-05-27T00:00:00Z,...,2021-06-10T00:00:00Z,2021-08-05T00:00:00Z,2021-06-24T00:00:00Z,2021-07-29T00:00:00Z,2021-08-05T00:00:00Z,2021-06-30T00:00:00Z,2021-07-15T00:00:00Z,2021-07-29T00:00:00Z,2021-07-22T00:00:00Z,2021-08-05T00:00:00Z
statistics,"{'usernames': 16322217, 'passwords': 16305095,...","{'usernames': 197675, 'passwords': 197768, 'em...","{'usernames': 2698, 'passwords': 4733278, 'ema...","{'usernames': 3198793, 'passwords': 3202109, '...","{'usernames': 111577, 'passwords': 697834, 'em...","{'usernames': 121453, 'passwords': 563927, 'em...","{'usernames': 10, 'passwords': 266358, 'emails...","{'usernames': 32521, 'passwords': 250505, 'ema...","{'usernames': 0, 'passwords': 449792, 'emails'...","{'usernames': 0, 'passwords': 179736, 'emails'...",...,"{'usernames': 1149613, 'passwords': 1108013, '...","{'usernames': 0, 'passwords': 16299, 'emails':...","{'usernames': 156123, 'passwords': 172352, 'em...","{'usernames': 271107, 'passwords': 104932, 'em...","{'usernames': 108984, 'passwords': 0, 'emails'...","{'usernames': 336911, 'passwords': 337164, 'em...","{'usernames': 255666, 'passwords': 4758916, 'e...","{'usernames': 0, 'passwords': 397, 'emails': 9...","{'usernames': 0, 'passwords': 432409, 'emails'...","{'usernames': 273666, 'passwords': 334332, 'em..."


In [19]:
res3 = res2.transpose()
res3

,breachId,site,recordsCount,description,publishDate,statistics
37398,37398,yam.com,16322233,"In 2013, the Taiwanese news website Yam was al...",2021-05-27T00:00:00Z,"{'usernames': 16322217, 'passwords': 16305095,..."
37428,37428,pearldrummersforums.com,197776,"At an unknown date, the online percussion site...",2021-05-13T00:00:00Z,"{'usernames': 197675, 'passwords': 197768, 'em..."
37458,37458,descomplica.com.br,4779635,"In March 2021, the Brazilian online education ...",2021-05-13T00:00:00Z,"{'usernames': 2698, 'passwords': 4733278, 'ema..."
37502,37502,swordfantasy.com,3202149,"At an unknown date, the forum for SwordFantasy...",2021-05-20T00:00:00Z,"{'usernames': 3198793, 'passwords': 3202109, '..."
37504,37504,neo24.pl,699115,"At an unknown date, the Polish online electron...",2021-06-30T00:00:00Z,"{'usernames': 111577, 'passwords': 697834, 'em..."
37509,37509,printi.com.br,563940,"At an unknown date, the Brazilian B2B printing...",2021-05-20T00:00:00Z,"{'usernames': 121453, 'passwords': 563927, 'em..."
37510,37510,ajarn.com,541191,"At an unknown date, the Thai English learning ...",2021-06-10T00:00:00Z,"{'usernames': 10, 'passwords': 266358, 'emails..."
37540,37540,wibdesign.de,250505,"At an unconfirmed date, the German web design ...",2021-07-29T00:00:00Z,"{'usernames': 32521, 'passwords': 250505, 'ema..."
37546,37546,piggyback.com,449792,"In July 2020, an online list of users of the g...",2021-05-20T00:00:00Z,"{'usernames': 0, 'passwords': 449792, 'emails'..."
37550,37550,eduever.com,187506,"At an unknown date, the Korean education websi...",2021-05-27T00:00:00Z,"{'usernames': 0, 'passwords': 179736, 'emails'..."


In [22]:
res3[['publishDate','site', 'description']]

,publishDate,site,description
37398,2021-05-27T00:00:00Z,yam.com,"In 2013, the Taiwanese news website Yam was al..."
37428,2021-05-13T00:00:00Z,pearldrummersforums.com,"At an unknown date, the online percussion site..."
37458,2021-05-13T00:00:00Z,descomplica.com.br,"In March 2021, the Brazilian online education ..."
37502,2021-05-20T00:00:00Z,swordfantasy.com,"At an unknown date, the forum for SwordFantasy..."
37504,2021-06-30T00:00:00Z,neo24.pl,"At an unknown date, the Polish online electron..."
37509,2021-05-20T00:00:00Z,printi.com.br,"At an unknown date, the Brazilian B2B printing..."
37510,2021-06-10T00:00:00Z,ajarn.com,"At an unknown date, the Thai English learning ..."
37540,2021-07-29T00:00:00Z,wibdesign.de,"At an unconfirmed date, the German web design ..."
37546,2021-05-20T00:00:00Z,piggyback.com,"In July 2020, an online list of users of the g..."
37550,2021-05-27T00:00:00Z,eduever.com,"At an unknown date, the Korean education websi..."
